# Pvsamv1Tools Examples

These examples demonstrate how to use PySAM's `Pvsamv1Tools` to size a system and access module and inverter parameters from the System Advisor Model (SAM) libraries, and how to use `ResourceTools` to download a weather file from the NREL National Solar Radiation Databse (NSRDB) and use it for a `Pvsamv1` simulation.

In [2]:
import PySAM
import PySAM.Pvsamv1 as pv
import PySAM.Pvsamv1Tools as pvtools
import PySAM.ResourceTools as resourcetools
print(PySAM.__version__)

7.0.0


## Use ResourceTools to download a weather file: FetchResourceFiles()

This function downloads weather files from the NREL National Solar Radiation Database (NSRDB). In this example, we are downloading a file for a single location and storing it in the current folder to use in the next example.

Register on the NREL Developer Network to get an NREL API key: https://developer.nrel.gov/signup/ and replace `DEMO_KEY` and `test_email.email.org` with your key and email address. `DEMO_KEY` only allows limited access to the API for testing.

In [3]:
nsrdb_get = resourcetools.FetchResourceFiles( tech='solar', nrel_api_key='DEMO_KEY', nrel_api_email='test_email@email.org', resource_dir='.' )
lat_lon = (-105.1,39.7)
nsrdb_get.fetch([lat_lon]) # this can be a list of lat_lon to download files for multiple locations
wf_path = nsrdb_get.resource_file_paths_dict[lat_lon]
print(wf_path)


Starting data download for solar using 1 thread workers.
Getting list of available NSRDB files for 39.7, -105.1.
List of available data saved to ./nsrdb_data_query_response_39.7_-105.1.json.
https://developer.nrel.gov/api/nsrdb/v2/solar/psm3-tmy-download.csv?names=tmy&wkt=POINT%28-105.1+39.7%29&interval=60&api_key=DEMO_KEY&email=test_email@email.org&utc=false
Success! File downloaded to ./nsrdb_39.7_-105.1_psm3-tmy_60_tmy.csv.
./nsrdb_39.7_-105.1_psm3-tmy_60_tmy.csv


## Run a simulation for the Default Detailed PV - Single Owner configuration

Run a simulation for the defaults using the weather file we just downloaded to get a baseline annual output to compare with the next set of examples.

See https://nrel-pysam.readthedocs.io/en/main/sam-configurations.html#detailed-pv-model-single-owner for a description of the configuration. You can use the approach in this example for other PV and PV-Battery configurations.

In [4]:
model = pv.default('FlatPlatePVSingleOwner')
model.SolarResource.solar_resource_file = wf_path
model.value('analysis_period', 1)
model.execute()
print(model.Outputs.annual_energy)

199255743.0945314


## Use Pvsamv1Tools to size the system: size_electrical_parameters()

This function sizes the PV system given a target system capacity in kW, target DC/AC ratio with optional maximum inverter DC voltage, inverter number of inputs (up to four), and maximum number of combiner boxes. (Note that SAM's PV sizing code on the System Design page does not account for the number of combiner boxes.)

For this example, we are sizing a 130,000 kWdc system with a DC/AC ratio of 1.2.

In [5]:
n_strings, n_combiners, n_inverters, calculated_system_capacity = pvtools.size_electrical_parameters(model, 130000, 1.2)
print(calculated_system_capacity)
print(n_strings)
print(n_inverters)
print(n_combiners)
print(model.value('inverter_count'))
model.execute()
print(model.Outputs.annual_energy)

130003.538112
8748
43
None
43.0
265107538.59492677


## Choose a module from SAM's CEC Module library: set_cec_module_library_selection()

This function downloads the latest version of the CSV file that stores SAM's CEC module library parameters from the SAM GitHub repository, and then sets the parameters for the Pvsamv1 `CECPerformanceModelWithModuleDatabase` group to values for the appropriate module from the library.

This requires that you look up the module name in the "Name" column of the CSV file for the module you want to select. In this example, we are downloading parameters for a module with the name "AIMS POWER PV555MONO".

https://github.com/NREL/SAM/blob/patch/deploy/libraries/CEC%20Modules.csv

In [6]:
model.value('module_model',1)
mod_dict = pvtools.set_cec_module_library_selection(model, "AIMS Power PV555MONO")
print(mod_dict)
print(model.value("cec_v_mp_ref"))

This function does not set the bifaciality of a module based on the bifacial based on the lib_is_bifacial value. Please change cec_is_bifacial to 1 to make the module bifacial.
{'lib_manufacturer': {'AIMS Power PV555MONO': 'AIMS Power'}, 'cec_material': {'AIMS Power PV555MONO': 'Mono-c-Si'}, 'lib_is_bifacial': {'AIMS Power PV555MONO': 0}, 'Unnamed: 4': {'AIMS Power PV555MONO': 554.673}, 'Unnamed: 5': {'AIMS Power PV555MONO': 522.2}, 'cec_area': {'AIMS Power PV555MONO': 2.61}, 'lib_length': {'AIMS Power PV555MONO': nan}, 'lib_width': {'AIMS Power PV555MONO': nan}, 'cec_n_s': {'AIMS Power PV555MONO': 72}, 'cec_i_sc_ref': {'AIMS Power PV555MONO': 13.74}, 'cec_v_oc_ref': {'AIMS Power PV555MONO': 49.9}, 'cec_i_mp_ref': {'AIMS Power PV555MONO': 12.81}, 'cec_v_mp_ref': {'AIMS Power PV555MONO': 43.3}, 'cec_alpha_sc': {'AIMS Power PV555MONO': 0.0038472}, 'cec_beta_oc': {'AIMS Power PV555MONO': -0.125249}, 'cec_t_noct': {'AIMS Power PV555MONO': 43.3}, 'cec_a_ref': {'AIMS Power PV555MONO': 1.8559

## Choose an inverter from SAM's CEC Inverter library: set_cec_inverter_library_selection()

This function downloads the latest version of the CSV file that stores SAM's CEC inverter library parameters from the SAM GitHub repository, and then sets the parameters for the Pvsamv1 `InverterCECDatabase` group to values for the appropriate inverter from the library.

This requires that you look up the inverter name in the "Name" column of the CSV file for the inverter you want to select. In this example, we are downloading parameters for an inverter with the name "NEXTRACKER LLC: NXPCS3340K-HUD [800V]".

https://github.com/NREL/SAM/blob/patch/deploy/libraries/CEC%2Inverers.csv

In [7]:
inv_dict = pvtools.set_cec_inverter_library_selection(model, "NEXTRACKER LLC: NXPCS3340K-HUD [800V]")
print(inv_dict)
print(model.value("inv_snl_paco"))

This function does not see the number of MPPT inputs for the inverter. Please see inv_num_mppt to set the number of mppt inputs for an inverter.
1  inverters selected. Name of 1st entry:  NEXTRACKER LLC: NXPCS3340K-HUD [800V]
{'inv_snl_ac_voltage': {'NEXTRACKER LLC: NXPCS3340K-HUD [800V]': 800}, 'inv_snl_pso': {'NEXTRACKER LLC: NXPCS3340K-HUD [800V]': 7431.25}, 'inv_snl_paco': {'NEXTRACKER LLC: NXPCS3340K-HUD [800V]': 3347800.0}, 'inv_snl_pdco': {'NEXTRACKER LLC: NXPCS3340K-HUD [800V]': 3408990.0}, 'inv_snl_vdco': {'NEXTRACKER LLC: NXPCS3340K-HUD [800V]': 1350.0}, 'inv_snl_c0': {'NEXTRACKER LLC: NXPCS3340K-HUD [800V]': -3.23937e-09}, 'inv_snl_c1': {'NEXTRACKER LLC: NXPCS3340K-HUD [800V]': 1.91598e-06}, 'inv_snl_c2': {'NEXTRACKER LLC: NXPCS3340K-HUD [800V]': 0.000220482}, 'inv_snl_c3': {'NEXTRACKER LLC: NXPCS3340K-HUD [800V]': -7.56066e-05}, 'inv_snl_pnt': {'NEXTRACKER LLC: NXPCS3340K-HUD [800V]': 443.65}, 'inv_snl_vdcmax': {'NEXTRACKER LLC: NXPCS3340K-HUD [800V]': 1500.0}, 'inv_snl_idc

## Run Pvsamv1 with new module and inverter

In [8]:
model.value('inverter_model', 0)
n_strings, n_combiners, n_inverters, calculated_system_capacity = pvtools.size_electrical_parameters(model, 130000, 1.2)
print(calculated_system_capacity)
print(n_strings)
print(n_inverters)
print(model.value('inverter_count'))
print(model.value('cec_v_mp_ref'))
print(model.value('cec_i_mp_ref'))
print(model.value('subarray1_modules_per_string'))
model.execute()
print(model.Outputs.annual_energy)

129993.16428
7812
32
32.0
43.3
12.81
30.0
264303340.1261947
